In [1]:
import wfdb
from wfdb import processing
import pandas as pd
import numpy as np
import re
import os
from datetime import timedelta

import requests
from bs4 import BeautifulSoup

import plotly.graph_objs as go
import plotly.offline as pyo

# Use the signal for filters
from scipy.signal import butter, filtfilt, iirnotch
from scipy.signal import welch, medfilt


In [2]:
# Base path to download the patient overview
base_url = "https://physionet.org/content/i-care/2.1/training/"
records_url = base_url + "RECORDS"

# Base directory path for the PhysioNet database
physionet_db_path = "i-care/2.1/training"

patient_numbers = []

In [3]:
destination_folder = "data"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

In [4]:
# Download the RECORDS file to get the list of patient numbers
response = requests.get(records_url)
if response.status_code == 200:
    # The RECORDS file is a plain text file, so split it by lines to get the patient numbers
    patient_numbers_html = response.text
else:
    print("Failed to download the RECORDS file.")

In [5]:
# Parse the HTML content
soup = BeautifulSoup(patient_numbers_html, 'html.parser')

# Find the <pre> tag with class "plain" and the <code> tag within it
code_tag = soup.find('pre', {'class': 'plain'}).find('code')

# Extract the content of the <code> tag
patient_records = code_tag.text if code_tag else ''

# Split the content into individual records (one per line)
patient_records_list = patient_records.splitlines()

# Clean the list by removing any empty strings
patient_records_list = [record.strip() for record in patient_records_list if record.strip()]

# Clean up the patient numbers by removing 'training/' and the trailing '/'
patient_numbers = [record.split('/')[1] for record in patient_records_list]

# Print the cleaned patient numbers to verify
print(patient_numbers)

['0284', '0286', '0296', '0299', '0303', '0306', '0311', '0312', '0313', '0316', '0319', '0320', '0326', '0328', '0332', '0334', '0335', '0337', '0340', '0341', '0342', '0344', '0346', '0347', '0348', '0349', '0350', '0351', '0352', '0353', '0354', '0355', '0356', '0357', '0358', '0359', '0360', '0361', '0362', '0363', '0364', '0365', '0366', '0367', '0368', '0369', '0370', '0371', '0372', '0373', '0375', '0376', '0377', '0378', '0379', '0380', '0382', '0383', '0384', '0385', '0387', '0389', '0390', '0391', '0392', '0394', '0395', '0396', '0397', '0398', '0399', '0400', '0402', '0403', '0404', '0405', '0406', '0407', '0409', '0410', '0411', '0412', '0413', '0414', '0415', '0416', '0417', '0418', '0419', '0420', '0421', '0422', '0423', '0424', '0426', '0427', '0428', '0429', '0430', '0431', '0432', '0433', '0434', '0435', '0436', '0437', '0438', '0439', '0440', '0441', '0442', '0443', '0444', '0445', '0446', '0447', '0448', '0450', '0451', '0452', '0453', '0455', '0456', '0457', '0458',

In [6]:
# Define the total number of segments based on `hours_included` in 5-minute intervals
hours_included = 22
total_segments = (hours_included * 60) // 5

# Define the columns by including `Mean_HR`, `HRV_SDNN`, `LF_Power`, `HF_Power`, and `LF/HF_Ratio` for each segment
columns = ["Patient_ID"] + [
    f"Segment_{i+1}_Mean_HR" if j == 0 else 
    f"Segment_{i+1}_HRV_SDNN" if j == 1 else 
    f"Segment_{i+1}_LF_Power" if j == 2 else 
    f"Segment_{i+1}_HF_Power" if j == 3 else 
    f"Segment_{i+1}_LF_HF_Ratio"
    for i in range(total_segments) for j in range(5)
]

# Create the DataFrame with specified columns
final_df = pd.DataFrame(columns=columns)

# Check the DataFrame structure
print(final_df.head())



Empty DataFrame
Columns: [Patient_ID, Segment_1_Mean_HR, Segment_1_HRV_SDNN, Segment_1_LF_Power, Segment_1_HF_Power, Segment_1_LF_HF_Ratio, Segment_2_Mean_HR, Segment_2_HRV_SDNN, Segment_2_LF_Power, Segment_2_HF_Power, Segment_2_LF_HF_Ratio, Segment_3_Mean_HR, Segment_3_HRV_SDNN, Segment_3_LF_Power, Segment_3_HF_Power, Segment_3_LF_HF_Ratio, Segment_4_Mean_HR, Segment_4_HRV_SDNN, Segment_4_LF_Power, Segment_4_HF_Power, Segment_4_LF_HF_Ratio, Segment_5_Mean_HR, Segment_5_HRV_SDNN, Segment_5_LF_Power, Segment_5_HF_Power, Segment_5_LF_HF_Ratio, Segment_6_Mean_HR, Segment_6_HRV_SDNN, Segment_6_LF_Power, Segment_6_HF_Power, Segment_6_LF_HF_Ratio, Segment_7_Mean_HR, Segment_7_HRV_SDNN, Segment_7_LF_Power, Segment_7_HF_Power, Segment_7_LF_HF_Ratio, Segment_8_Mean_HR, Segment_8_HRV_SDNN, Segment_8_LF_Power, Segment_8_HF_Power, Segment_8_LF_HF_Ratio, Segment_9_Mean_HR, Segment_9_HRV_SDNN, Segment_9_LF_Power, Segment_9_HF_Power, Segment_9_LF_HF_Ratio, Segment_10_Mean_HR, Segment_10_HRV_SDNN, Seg

In [7]:
def apply_high_low_filter(signal, sampling_rate, utility_freq):
    # Step 1: High-pass filter to remove baseline drift (1 Hz cutoff)
    b_high, a_high = butter(2, 1 / (sampling_rate / 2), btype='high')
    filtered_signal = filtfilt(b_high, a_high, signal)

    # Step 2: Notch filter to remove utility frequency noise (50 Hz or 60 Hz)
    Q = 30.0  # Quality factor for notch filter
    notch_freq = utility_freq / (sampling_rate / 2)
    b_notch, a_notch = iirnotch(notch_freq, Q)
    filtered_signal = filtfilt(b_notch, a_notch, filtered_signal)

    # Step 3: Stronger low-pass filter to reduce high-frequency noise (40 Hz cutoff, 4th order)
    b_low, a_low = butter(4, 40 / (sampling_rate / 2), btype='low')
    filtered_signal = filtfilt(b_low, a_low, filtered_signal)

    # Step 4: Apply a median filter to remove spikes
    filtered_signal = medfilt(filtered_signal, kernel_size=5)

    return filtered_signal

In [ ]:
from scipy.signal import welch
from datetime import timedelta
import numpy as np
from wfdb import processing

def calculate_hr_hrv_lf_hf_ratio(signal, sampling_rate, local_start_time, total_samples):
    print(f"Sampling: {sampling_rate}")
    # Set thresholds and frequency bands
    hr_threshold = (30, 200)
    hrv_threshold = (0, 300)
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    # Calculate the duration to the next full 5-minute boundary from local_start_time
    first_segment_duration = timedelta(minutes=5) - timedelta(seconds=local_start_time.total_seconds() % 300)
    first_segment_duration_seconds = first_segment_duration.total_seconds()

    # Calculate the sample index to end the first segment
    first_segment_end_index = int(first_segment_duration_seconds * sampling_rate)
    first_segment_signal = signal[:first_segment_end_index]

    # Initialize XQRS for the first segment and detect QRS complexes
    xqrs = processing.XQRS(sig=first_segment_signal, fs=sampling_rate)
    xqrs.detect(learn=False, verbose=False)
    qrs_indices = np.array(xqrs.qrs_inds)  # Indices of detected R-peaks in sample points
    r_wave_times = qrs_indices / sampling_rate

    # Initialize lists to store HR, HRV, LF, HF values, LF/HF ratios, and QRS indices
    hr_values, hrv_values, lf_powers, hf_powers, lfhf_ratios = [], [], [], [], []
    all_qrs_indices = []  # List to accumulate QRS indices from each segment
    segment_start_indices = [0]

    # Append QRS indices for the first segment
    all_qrs_indices.extend(qrs_indices)

    # Process the shortened first segment
    first_segment_indices = r_wave_times < first_segment_duration_seconds
    first_segment_rr_intervals = np.diff(r_wave_times[first_segment_indices]) * 1000  # Convert to milliseconds

    # Calculate HR, HRV, LF, HF, and LF/HF for the first segment
    if len(first_segment_rr_intervals) > 0:
        mean_hr = int(round(processing.calc_mean_hr(first_segment_rr_intervals / 1000, rr_units="seconds")))
        hrv = int(round(np.std(first_segment_rr_intervals)))

        # Apply threshold conditions
        mean_hr = mean_hr if hr_threshold[0] <= mean_hr <= hr_threshold[1] else np.nan
        hrv = hrv if hrv_threshold[0] <= hrv <= hrv_threshold[1] else np.nan

        # Calculate LF and HF power
        freqs, psd = welch(first_segment_rr_intervals, fs=1.0, nperseg=len(first_segment_rr_intervals))
        lf_power = int(np.trapezoid(psd[(freqs >= lf_band[0]) & (freqs < lf_band[1])], freqs[(freqs >= lf_band[0]) & (freqs < lf_band[1])]))
        hf_power = int(np.trapezoid(psd[(freqs >= hf_band[0]) & (freqs < hf_band[1])], freqs[(freqs >= hf_band[0]) & (freqs < hf_band[1])]))
        lfhf_ratio = round((lf_power / hf_power), 2) if hf_power > 0 else np.nan
    else:
        mean_hr, hrv, lf_power, hf_power, lfhf_ratio = np.nan, np.nan, np.nan, np.nan, np.nan

    # Append results for the first segment
    hr_values.append(mean_hr)
    hrv_values.append(hrv)
    lf_powers.append(lf_power)
    hf_powers.append(hf_power)
    lfhf_ratios.append(lfhf_ratio)

    # Process remaining full 5-minute segments
    current_time = first_segment_duration_seconds
    num_segments = int((total_samples / sampling_rate - current_time) // 300)  # Remaining time for full segments
    for _ in range(num_segments):
        segment_start_index = int(current_time * sampling_rate)
        segment_start_indices.append(segment_start_index)
        
        segment_end = current_time + 300
        segment_signal = signal[segment_start_index:int(segment_end * sampling_rate)]
        
        # Initialize XQRS for the current segment
        xqrs = processing.XQRS(sig=segment_signal, fs=sampling_rate)
        xqrs.detect(learn=False, verbose=False)
        qrs_indices = np.array(xqrs.qrs_inds)
        all_qrs_indices.extend(qrs_indices + segment_start_index)  # Adjust indices to the full signal context
        selected_r_wave_times = (qrs_indices + segment_start_index) / sampling_rate  # Convert to seconds with offset
        segment_rr_intervals = np.diff(selected_r_wave_times) * 1000  # Convert RR intervals to milliseconds

        # Calculate HR, HRV, LF, HF, and LF/HF for the segment
        if len(segment_rr_intervals) > 0:
            mean_hr = int(round(processing.calc_mean_hr(segment_rr_intervals / 1000, rr_units="seconds")))
            hrv = int(round(np.std(segment_rr_intervals)))

            mean_hr = mean_hr if hr_threshold[0] <= mean_hr <= hr_threshold[1] else np.nan
            hrv = hrv if hrv_threshold[0] <= hrv <= hrv_threshold[1] else np.nan

            freqs, psd = welch(segment_rr_intervals, fs=1.0, nperseg=len(segment_rr_intervals))
            lf_power = int(np.trapezoid(psd[(freqs >= lf_band[0]) & (freqs < lf_band[1])], freqs[(freqs >= lf_band[0]) & (freqs < lf_band[1])]))
            hf_power = int(np.trapezoid(psd[(freqs >= hf_band[0]) & (freqs < hf_band[1])], freqs[(freqs >= hf_band[0]) & (freqs < hf_band[1])]))
            lfhf_ratio = round((lf_power / hf_power), 2) if hf_power > 0 else np.nan
        else:
            mean_hr, hrv, lf_power, hf_power, lfhf_ratio = np.nan, np.nan, np.nan, np.nan, np.nan

        # Append the results for each segment
        hr_values.append(mean_hr)
        hrv_values.append(hrv)
        lf_powers.append(lf_power)
        hf_powers.append(hf_power)
        lfhf_ratios.append(lfhf_ratio)

        # Move to the next 5-minute segment
        current_time = segment_end

    # Convert all_qrs_indices to a numpy array if needed
    all_qrs_indices = np.array(all_qrs_indices)

    return hr_values, hrv_values, lf_powers, hf_powers, lfhf_ratios, all_qrs_indices, segment_start_indices


In [9]:
def transfer_to_final_structure(patient_id, hr_hrv_df, final_df):
    # Create an empty row with NaN values for all segments
    row_data = {"Patient_ID": patient_id}
    for column in final_df.columns:
        if column != "Patient_ID":
            row_data[column] = np.nan

    # If hr_hrv_df has data, fill in the corresponding HR, HRV, LF, HF, and LF/HF values
    if not hr_hrv_df.empty:
        for i in range(len(hr_hrv_df)):
            hr_column = f"Segment_{i+1}_Mean_HR"
            hrv_column = f"Segment_{i+1}_HRV_SDNN"
            lf_column = f"Segment_{i+1}_LF_Power"
            hf_column = f"Segment_{i+1}_HF_Power"
            lfhf_ratio_column = f"Segment_{i+1}_LF_HF_Ratio"
            
            # Assign values using column names that match hr_hrv_df
            row_data[hr_column] = hr_hrv_df.at[i, "Mean HR (bpm)"]
            row_data[hrv_column] = hr_hrv_df.at[i, "HRV (SDNN) (ms)"]
            row_data[lf_column] = hr_hrv_df.at[i, "LF Power (ms^2)"]
            row_data[hf_column] = hr_hrv_df.at[i, "HF Power (ms^2)"]
            row_data[lfhf_ratio_column] = hr_hrv_df.at[i, "LF/HF Ratio"]

    # Convert row_data dictionary to a DataFrame and concatenate with final_df
    new_row_df = pd.DataFrame([row_data])
    final_df = pd.concat([final_df, new_row_df], ignore_index=True)
    return final_df

In [10]:
def get_patient_records(patient_number):
    # Construct the full URL to the RECORDS file
    file_url = f"{base_url}/{patient_number}/RECORDS"

    # Fetch the URL content
    response = requests.get(file_url)
    if response.status_code == 200:
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the text content of the RECORDS file
        patient_records = soup.text  # Plain text response for RECORDS

        # Split the content into lines and filter for `_ECG` files with last 3 digits <= 023
        ecg_files = [
            line for line in patient_records.splitlines()
            if line.endswith("_ECG") and int(line.split("_")[-2]) <= 5
        ]

        return ecg_files
    else:
        print(f"Failed to retrieve the file. Status code: {response.status_code}")

In [ ]:
# Get the first 20 entries
first_20_patients = ['0432'] # patient_numbers[:2]

total_patients = len(first_20_patients)

for i, patient_number in enumerate(first_20_patients, start=1):
    print("===================================================")
    print(f"Starting Patient {i} of {total_patients} (Patient number {patient_number})")
    print("===================================================")
    ecg_files = get_patient_records(patient_number)


    # Initialize a DataFrame for a full 24-hour period, with 5-minute segments
    total_duration_hours = 24
    segment_duration = timedelta(minutes=5)
    total_segments = (total_duration_hours * 60) // 5

    # Create the full DataFrame with NaN values initially
    hr_hrv_df = pd.DataFrame({
        "Start Time": [timedelta(minutes=5 * i) for i in range(total_segments)],
        "Mean HR (bpm)": [np.nan] * total_segments,
        "HRV (SDNN) (ms)": [np.nan] * total_segments
    })

    for ecg_file in ecg_files:

        # Construct the file's record name (without extension)
        record_name = ecg_file
        
        try:
            # Load the record using PhysioNet-specific directory
            record = wfdb.rdrecord(record_name, pn_dir=f"{physionet_db_path}/{patient_number}")
            print(f"Successfully loaded: {ecg_file}")
        except Exception as e:
            print(f"Failed to load {ecg_file}: {e}")

        # Get metadata of the signal
        patient_number = record.record_name.split("_")[0]
        sampling_rate = record.fs
        signal_len = record.sig_len
        utility_freq = int(record.comments[0].split(": ")[1])
        start_time = pd.to_timedelta(record.comments[1].split(": ")[1])
        end_time = pd.to_timedelta(record.comments[2].split(": ")[1])

        print(sampling_rate)


        # Access the signal data (numpy array) for the second lead
        if record.p_signal.shape[1] > 1:  # Check if there are multiple leads
            ecg_signal_data = record.p_signal[:, 0].flatten()  # Select the second lead
            print("Loaded second lead.")
        else:
            print("Only one lead available in this record.")
            # Access the signal data (numpy array)
            ecg_signal_data = record.p_signal.flatten()

        

        filtered_signal = apply_high_low_filter(ecg_signal_data, sampling_rate, utility_freq)

        # Call the updated function to calculate HR, HRV, LF, HF, and LF/HF ratio
        hr_values, hrv_values, lf_values, hf_values, lfhf_ratios, qrs_indices, segment_start_points = calculate_hr_hrv_lf_hf_ratio(
            filtered_signal, sampling_rate, start_time, signal_len
        )
        print(f"hr_values: {hr_values}")
        print(f"hrv_values: {hrv_values}")
        print(f"lf_values: {lf_values}")
        print(f"hf_values: {hf_values}")
        print(f"lfhf_ratios: {lfhf_ratios}")
        # Find the nearest 5-minute interval for `start_time`
        start_index = hr_hrv_df[hr_hrv_df["Start Time"] <= start_time].last_valid_index()

        # Populate `hr_hrv_df` starting from `start_index`
        for i, (hr, hrv, lf, hf, lfhf) in enumerate(zip(hr_values, hrv_values, lf_values, hf_values, lfhf_ratios)):
            if start_index + i < len(hr_hrv_df):  # Ensure we don't go out of bounds
                hr_hrv_df.at[start_index + i, "Mean HR (bpm)"] = hr
                hr_hrv_df.at[start_index + i, "HRV (SDNN) (ms)"] = hrv
                hr_hrv_df.at[start_index + i, "LF Power (ms^2)"] = lf
                hr_hrv_df.at[start_index + i, "HF Power (ms^2)"] = hf
                hr_hrv_df.at[start_index + i, "LF/HF Ratio"] = lfhf


    # After processing all files for a patient, call this function
    final_df = transfer_to_final_structure(patient_number, hr_hrv_df, final_df)

Starting Patient 1 of 1 (Patient number 0432)
Successfully loaded: 0432_001_005_ECG
Loaded second lead.
hr_values: [87, 184, nan, 194, 200, 105]
hrv_values: [257, 21, 58, 63, 59, nan]
lf_values: [165, 0, 0, 0, 0, 4574]
hf_values: [19876, 0, 2079, 3929, 2079, 6959]
lfhf_ratios: [0.01, nan, 0.0, 0.0, 0.0, 0.66]


C:\Users\kentf\AppData\Local\Temp\ipykernel_7596\744392416.py:26: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [12]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Create the Plotly plot for the ECG Signal
trace_signal = go.Scatter(
    y=filtered_signal,
    mode='lines',
    name='ECG Signal'
)

# Create the Plotly plot for the R-Peaks
r_peaks_y = [filtered_signal[i] for i in qrs_indices]  # Get the amplitudes of the R-peaks
trace_r_peaks = go.Scatter(
    x=qrs_indices,
    y=r_peaks_y,
    mode='markers',
    name='R-Peaks',
    marker=dict(color='red', size=8, symbol='x')  # Customize the appearance
)

# Create vertical line markers at the start of each 5-minute segment
segment_lines = [
    go.layout.Shape(
        type="line",
        x0=boundary,
        y0=min(filtered_signal),  # Start from minimum signal amplitude
        x1=boundary,
        y1=max(filtered_signal),  # End at maximum signal amplitude
        line=dict(color="blue", width=1, dash="dash"),
    ) for boundary in segment_start_points
]

# Define the layout with added shapes
layout = go.Layout(
    title='Scrollable ECG Signal with R-Peaks and 5-Minute Segment Lines',
    xaxis=dict(
        title='Time (samples)',
        rangeslider=dict(visible=True),  # Add a range slider to make it scrollable
    ),
    yaxis=dict(
        title='Amplitude',
    ),
    shapes=segment_lines,  # Use segment_lines instead of shapes
    width=1200,  # Set a wider plot
    height=500
)

# Create the figure and add both traces
fig = go.Figure(data=[trace_signal, trace_r_peaks], layout=layout)

# Show the plot in an interactive window
pyo.plot(fig, filename='scrollable_ecg_with_r_peaks_segments.html')


'scrollable_ecg_with_r_peaks_segments.html'

In [13]:
final_df.to_csv('data/hr_hrv_df.csv', index=False)

In [14]:
print("Segment boundaries (in samples):", segment_start_points)

Segment boundaries (in samples): [0, 31744, 108544, 185344, 262144, 338944]
